# Cleaning Cyclistic Bike-Share Data
This notebook has the purpose of loading, concatenating, and cleaning the data as a part of the Google Data Analytics Capstone project (Case Study 1). The result will be a clean and usable single file for analysis.

### Analysis Task
As a Junior Analyst at the fictitious company *Cyclistic*, I have been tasked with dsicovering the differences between "Casual Riders" and "Annual Members". This is with the goal in mind of converting casual riders to annual members, as they generate more revenue. These findings will be explored in a separate notebook focused on analysis - where the analysis will be done on the output file of this notebook. These goals should be kept in mind during the cleaning process.

### Data Source

The historical data used for my analysis can be found [here](https://divvy-tripdata.s3.amazonaws.com/index.html). It is an archive of the company's transactions, found in zip files, from the year 2013 until the present. Some files are organized by year, quarter, or month - so it must be noted and organized in analysis accordingly. For the purpose of this assignment, I have taken the previous years data (July, 2020 - June, 2021). There are 12 .csv files within the zip folders - one for each month - that must be concated together for analysis. 

This is real-world public data that was made available by *Motivate International Inc.* under this [license](https://www.divvybikes.com/data-license-agreement)

**Note:** Data-privacy issues prohibit the use of personally identifiable information. Therefore it is impossible to connect pass purchase to credit card numbers, for the purpose of determining if the riders live within the Cyclistic service area, or if they have purchased multiple single passes. This may slightly skew data (as it will be unknown if a single person has purchased multiple day passes where they may be more inclined to buy the annual pass), but should not affect analysis too much, as there is such a large data set that we are working with, this number of affected riders will be insignifcant to final analysis. 

### Data Cleaning Tasks
Here is a list of all the completed data cleaning and manipulation tasks completed below:
- Concat files into one
- drop station ID columns (2)
- remove 209 duplicates (from `ride_id`)
- create columns for trip duration
- create column for trip duration in minutes
- remove entries with zero or negative trip duration
- create column for the weekday that trip starts
- convert `member_casual` and `weekday` columns to 'category' data type.
- Drop entries missing `end_lat` and/or `end_lng` values.
- Replace missing station names with 'Unknown'

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [2]:
# pandas environment
pd.options.display.float_format = '{:,.2f}'.format #Set to two decimal places

In [3]:
# Load first and last file for quick check
df_first = pd.read_csv('data/raw/202007-divvy-tripdata.csv')
df_last = pd.read_csv('data/raw/202106-divvy-tripdata.csv')

In [4]:
# Look at first five entries of first file
df_first.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,762198876D69004D,docked_bike,2020-07-09 15:22:02,2020-07-09 15:25:52,Ritchie Ct & Banks St,180.00,Wells St & Evergreen Ave,291.00,41.91,-87.63,41.91,-87.63,member
1,BEC9C9FBA0D4CF1B,docked_bike,2020-07-24 23:56:30,2020-07-25 00:20:17,Halsted St & Roscoe St,299.00,Broadway & Ridge Ave,461.00,41.94,-87.65,41.98,-87.66,member
2,D2FD8EA432C77EC1,docked_bike,2020-07-08 19:49:07,2020-07-08 19:56:22,Lake Shore Dr & Diversey Pkwy,329.00,Clark St & Wellington Ave,156.00,41.93,-87.64,41.94,-87.65,casual
3,54AE594E20B35881,docked_bike,2020-07-17 19:06:42,2020-07-17 19:27:38,LaSalle St & Illinois St,181.00,Clark St & Armitage Ave,94.00,41.89,-87.63,41.92,-87.64,casual
4,54025FDC7440B56F,docked_bike,2020-07-04 10:39:57,2020-07-04 10:45:05,Lake Shore Dr & North Blvd,268.00,Clark St & Schiller St,301.00,41.91,-87.63,41.91,-87.63,member


In [5]:
# Look at last five entries of last file
df_last.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
729590,CB282292CCFCE74F,electric_bike,2021-06-14 00:17:31,2021-06-14 00:56:46,Wells St & Polk St,SL-011,NaN,NaN,41.87,-87.63,41.88,-87.62,casual
729591,47BD346FAFB9BE6D,classic_bike,2021-06-30 17:35:10,2021-06-30 17:43:20,Clark St & Chicago Ave,13303,Kingsbury St & Kinzie St,KA1503000043,41.90,-87.63,41.89,-87.64,member
729592,52467C23D17C6AFE,classic_bike,2021-06-13 19:24:30,2021-06-13 19:34:11,Indiana Ave & 26th St,TA1307000005,State St & 19th St,SL-013,41.85,-87.62,41.86,-87.63,member
729593,7DF6D74420D7D9E6,electric_bike,2021-06-08 15:44:28,2021-06-08 16:15:01,Clark St & Chicago Ave,13303,NaN,NaN,41.90,-87.63,41.94,-87.71,member
729594,0C01F8BA99E512E5,electric_bike,2021-06-03 16:18:38,2021-06-03 16:47:49,Clark St & Chicago Ave,13303,NaN,NaN,41.90,-87.63,41.94,-87.71,member


In [6]:
# Compare column headers
df_first.columns == df_last.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [7]:
# Compare Data Types 
df_first.dtypes == df_last.dtypes

ride_id                True
rideable_type          True
started_at             True
ended_at               True
start_station_name     True
start_station_id      False
end_station_name       True
end_station_id        False
start_lat              True
start_lng              True
end_lat                True
end_lng                True
member_casual          True
dtype: bool

Notice that the `start_station_id` and `end_station_id` datatypes do not match. This is also seen in the 5 entries as each - it appears that at some point the company changed how they ID their stations. Seeing as this information is irrelevant to our analysis, we will drop these columns.

#### Concatenation
In this section, the 12 files will be concatenated into one. Each file will drop the station id columns, will check for matching headers, and checked for matching data types.

In [8]:
directory = 'data/raw'
df_all = pd.DataFrame() #empty DataFrame to populate

# Iterate through each file
for filename in os.listdir(directory):
    if filename.endswith('.csv'): #only want the CSV files
        fileloc = os.path.join(directory, filename)
        
        #open the next file
        with open(fileloc) as f:
            df_new = pd.read_csv(fileloc)
            df_new.drop(['start_station_id','end_station_id'], axis=1, inplace=True)
            print(filename + " has " + str(df_new.shape[0]) + " rows.")
            df_all = pd.concat([df_all, df_new])
            
        # Check for difference in columns
        columndiff = df_all.columns.symmetric_difference(df_new.columns)
        if (not columndiff.empty):
            print("", "Different Column Names for:", filename, columndiff, "", sep="\n")
        else:
            print("Column Headers Match!")
            
        # Check for difference in dtypes:
        if df_all.dtypes.to_list() == df_new.dtypes.to_list():
            print("Data Types Match!")
            
df_all.reset_index(drop=True, inplace=True) # resets the index of the large dataframe

print("All files concatenated into one DataFrame with {} rows.".format(df_all.shape[0]))

202007-divvy-tripdata.csv has 551480 rows.
Column Headers Match!
Data Types Match!
202008-divvy-tripdata.csv has 622361 rows.
Column Headers Match!
Data Types Match!
202009-divvy-tripdata.csv has 532958 rows.
Column Headers Match!
Data Types Match!
202010-divvy-tripdata.csv has 388653 rows.
Column Headers Match!
Data Types Match!
202011-divvy-tripdata.csv has 259716 rows.
Column Headers Match!
Data Types Match!
202012-divvy-tripdata.csv has 131573 rows.
Column Headers Match!
Data Types Match!
202101-divvy-tripdata.csv has 96834 rows.
Column Headers Match!
Data Types Match!
202102-divvy-tripdata.csv has 49622 rows.
Column Headers Match!
Data Types Match!
202103-divvy-tripdata.csv has 228496 rows.
Column Headers Match!
Data Types Match!
202104-divvy-tripdata.csv has 337230 rows.
Column Headers Match!
Data Types Match!
202105-divvy-tripdata.csv has 531633 rows.
Column Headers Match!
Data Types Match!
202106-divvy-tripdata.csv has 729595 rows.
Column Headers Match!
Data Types Match!
All fi

In [9]:
# Examine 5 random entries
df_all.sample(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
2926244,7FB5068833366CF4,classic_bike,2021-04-06 10:34:35,2021-04-06 11:01:00,Kimbark Ave & 53rd St,University Ave & 57th St,41.80,-87.59,41.79,-87.60,member
2815954,C7944A952B38C9BD,classic_bike,2021-03-19 00:13:37,2021-03-19 00:16:34,Sheffield Ave & Webster Ave,Racine Ave & Fullerton Ave,41.92,-87.65,41.93,-87.66,member
2312862,B45AD071FC00EBE9,docked_bike,2020-11-14 15:32:38,2020-11-14 15:52:23,Wood St & Milwaukee Ave,Sheffield Ave & Willow St,41.91,-87.67,41.91,-87.65,member
1488996,259B283A3167AF62,electric_bike,2020-09-27 16:38:07,2020-09-27 17:09:39,NaN,NaN,41.88,-87.70,41.86,-87.69,member
71399,D0B2231F880CC496,docked_bike,2020-07-24 19:48:14,2020-07-24 20:17:40,Clark St & Winnemac Ave,Sheridan Rd & Loyola Ave,41.97,-87.67,42.00,-87.66,member


In [10]:
# Check info of DataFrame
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4460151 entries, 0 to 4460150
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   end_station_name    object 
 6   start_lat           float64
 7   start_lng           float64
 8   end_lat             float64
 9   end_lng             float64
 10  member_casual       object 
dtypes: float64(4), object(7)
memory usage: 374.3+ MB


#### Cleaning

In [11]:
# Check for missing values
df_all.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    282068
end_station_name      315109
start_lat                  0
start_lng                  0
end_lat                 5286
end_lng                 5286
member_casual              0
dtype: int64

Notice that all the missing values come from columns that deal with location. This can be ignored for now, but will be addressed later. 

In [12]:
# Want to set ride_id column as index
## Check that all ride_id are unique
df_all.ride_id.nunique()

4459942

Notice that this number is less than the total number of entries! There must be some duplicate entries

In [13]:
# Check for duplicates
df_all.ride_id.duplicated().sum()

209

In [14]:
# Check some of these duplicate entries
df_all.loc[df_all[['ride_id']].duplicated()].sort_values('started_at') # sort by the start date

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
2381133,BE1F691E387D7072,docked_bike,2020-12-15 11:34:23,2020-11-25 08:00:15,Kingsbury St & Kinzie St,Canal St & Monroe St,41.89,-87.64,41.88,-87.64,member
2393756,325F6C4D5B72134A,docked_bike,2020-12-15 11:34:31,2020-11-25 08:20:28,Clarendon Ave & Junior Ter,Broadway & Wilson Ave,41.96,-87.65,41.97,-87.66,member
2446056,BB4697B59E3FF12A,docked_bike,2020-12-15 11:34:38,2020-11-25 08:09:43,Clark St & 9th St (AMLI),Dearborn St & Adams St,41.87,-87.63,41.88,-87.63,member
2425912,A180F0B9240F6660,docked_bike,2020-12-15 11:34:40,2020-11-25 08:38:56,California Ave & Francis Pl (Temp),Milwaukee Ave & Wabansia Ave,41.92,-87.70,41.91,-87.68,casual
2475615,5A0D169EFA5A7C4A,docked_bike,2020-12-15 11:34:51,2020-11-25 08:20:50,Kingsbury St & Erie St,Clinton St & Lake St,41.89,-87.64,41.89,-87.64,member
...,...,...,...,...,...,...,...,...,...,...,...
2380425,A08B4A7018A2C588,docked_bike,2020-12-15 12:21:18,2020-11-25 16:31:14,Paulina St & Flournoy St,Wells St & Polk St,41.87,-87.67,41.87,-87.63,member
2483631,332340BB61D42B3C,docked_bike,2020-12-15 12:21:37,2020-11-25 14:49:41,Bissell St & Armitage Ave,Clark St & Drummond Pl,41.92,-87.65,41.93,-87.64,casual
2404267,3C8888090D42A63F,docked_bike,2020-12-15 12:21:46,2020-11-25 17:38:06,Wood St & Taylor St (Temp),Laflin St & Cullerton St,41.87,-87.67,41.85,-87.66,member
2452141,165F7B547D1CFAC0,docked_bike,2020-12-15 12:21:50,2020-11-25 19:24:03,Clark St & Elm St,Fairbanks Ct & Grand Ave,41.90,-87.63,41.89,-87.62,member


Notice how all the duplicate entries start on December 15, 2020. They also end on November 25, 2020. Something must be wrong with the file they came from, as this difference in time isn't even possible. Before removing these duplicate files, let's quickly look at single duplicated `ride_id` values:

In [15]:
# Look at first duplicated ride ID
df_all.loc[df_all.ride_id == 'BE1F691E387D7072']

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
2187816,BE1F691E387D7072,docked_bike,2020-11-25 07:54:50,2020-11-25 08:00:15,Kingsbury St & Kinzie St,Canal St & Monroe St,41.89,-87.64,41.88,-87.64,member
2381133,BE1F691E387D7072,docked_bike,2020-12-15 11:34:23,2020-11-25 08:00:15,Kingsbury St & Kinzie St,Canal St & Monroe St,41.89,-87.64,41.88,-87.64,member


In [21]:
# Look at last duplicated ride ID
df_all.loc[df_all.ride_id == 'C763BA9D0F6ED827']

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
2354327,C763BA9D0F6ED827,docked_bike,2020-11-25 18:53:00,2020-11-25 19:13:39,Broadway & Barry Ave,Clark St & Armitage Ave,41.94,-87.64,41.92,-87.64,member
2376346,C763BA9D0F6ED827,docked_bike,2020-12-15 12:21:52,2020-11-25 19:13:39,Broadway & Barry Ave,Clark St & Armitage Ave,41.94,-87.64,41.92,-87.64,member


Just as expected, the entries are exact duplicates, aside from the `started_at` values. It is then safe to drop the second entry of each duplicate from the entire DataFrame.

In [23]:
# Drop duplicates
df_all.drop_duplicates(subset='ride_id', keep='first', inplace=True)

In [24]:
# Check that Successfully dropped duplicates
df_all.ride_id.duplicated().sum()

0

In [26]:
# Now we can set ride id as the index
df_all.set_index('ride_id', inplace=True)

Now that duplicates have been dealt with, and the DataFrame has been reindexed, let's look at converting data types to the proper format

In [28]:
# Convert started_at and ended_at to datetime format
from datetime import datetime

df_all['started_at'] = pd.to_datetime(df_all.started_at)
df_all['ended_at'] = pd.to_datetime(df_all.ended_at)

In [29]:
# Check
df_all.dtypes

rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
end_station_name              object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
dtype: object

In [39]:
# Create new column for the duration of the ride
df_all['duration'] = df_all['ended_at'] - df_all['started_at']
df_all.duration.describe()

count                      4459942
mean     0 days 00:25:09.251399233
std      0 days 04:58:42.225954644
min             -21 days +19:50:02
25%                0 days 00:07:31
50%                0 days 00:13:38
75%                0 days 00:25:08
max               38 days 20:24:09
Name: duration, dtype: object

In [40]:
# Add column for trip duration in minutes (easier for comparison during analysis)
df_all['duration_mins'] = df_all.duration.apply(lambda x: x.total_seconds()/60)

In [42]:
df_all.duration_mins.describe()

count   4,459,942.00
mean           25.15
std           298.70
min       -29,049.97
25%             7.52
50%            13.63
75%            25.13
max        55,944.15
Name: duration_mins, dtype: float64

In [50]:
# Now let's remove all entries with negative or zero ride durations
df_neg_dur = df_all.loc[df_all.duration_mins <= 0] #dataframe containing entries with negative or zero trip length
df_all.drop(df_neg_dur.index, axis=0, inplace=True)

In [51]:
# Check that proper entries were dropped
df_all.duration_mins.describe()

count   4,449,799.00
mean           26.30
std           241.57
min             0.02
25%             7.55
50%            13.68
75%            25.18
max        55,944.15
Name: duration_mins, dtype: float64

Great, now we only have entries with a ride length > 0 minutes. Now let's add a column that determines what day of the week the ride begins on. 

In [52]:
# get weekday
df_all['weekday'] = df_all['started_at'].apply(lambda x: x.day_name())

In [54]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4449799 entries, 762198876D69004D to 0C01F8BA99E512E5
Data columns (total 13 columns):
 #   Column              Dtype          
---  ------              -----          
 0   rideable_type       object         
 1   started_at          datetime64[ns] 
 2   ended_at            datetime64[ns] 
 3   start_station_name  object         
 4   end_station_name    object         
 5   start_lat           float64        
 6   start_lng           float64        
 7   end_lat             float64        
 8   end_lng             float64        
 9   member_casual       object         
 10  duration            timedelta64[ns]
 11  duration_mins       float64        
 12  weekday             object         
dtypes: datetime64[ns](2), float64(5), object(5), timedelta64[ns](1)
memory usage: 475.3+ MB


Now that we have added columns of interest, we should change the data type of `member_casual` and `weekday` to categorical. This will help reduce the file size.

In [55]:
# Convert weekday column to category datatype
df_all['weekday'] = df_all['weekday'].astype('category')

In [56]:
# Convert member_casual column to category datatype
df_all['member_casual'] = df_all['member_casual'].astype('category')

In [57]:
# Check
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4449799 entries, 762198876D69004D to 0C01F8BA99E512E5
Data columns (total 13 columns):
 #   Column              Dtype          
---  ------              -----          
 0   rideable_type       object         
 1   started_at          datetime64[ns] 
 2   ended_at            datetime64[ns] 
 3   start_station_name  object         
 4   end_station_name    object         
 5   start_lat           float64        
 6   start_lng           float64        
 7   end_lat             float64        
 8   end_lng             float64        
 9   member_casual       category       
 10  duration            timedelta64[ns]
 11  duration_mins       float64        
 12  weekday             category       
dtypes: category(2), datetime64[ns](2), float64(5), object(3), timedelta64[ns](1)
memory usage: 415.9+ MB


Now that all of that has been sorted out, let's move on to dealing with missing values!

In [58]:
# Check for missing values
df_all.isna().sum()

rideable_type              0
started_at                 0
ended_at                   0
start_station_name    282014
end_station_name      314821
start_lat                  0
start_lng                  0
end_lat                 5234
end_lng                 5234
member_casual              0
duration                   0
duration_mins              0
weekday                    0
dtype: int64

In [61]:
# Check how many rows contain 3 or more missing values
(df_all.isna().sum(axis=1) >2).sum()

5234

In [60]:
# Check how many unique values for latitude and longitude
print(df_all.start_lat.nunique())
print(df_all.start_lng.nunique())

322200
310110


This is alot of unique values, meaning that latitude and longitude have been recorded differently for different entries (perhaps more decimal places for some entries). This being said, it would be quite difficult to match latitude and longitude to station names in order to fill these missing values.

Instead, we will drop the entries with missing `end_lat` and `end_lng`. This is viable because there are only 5234 insatnces (out of 4449799 entries), and because it appears that all entries missing these values, are also missing the `end_station_name` - so matching to known stations is impossible.

We will also fill all missing station names with 'Unknown' (instead of dropping entries) should we want to use this information in later analysis. 

In [62]:
# Drop entries missing lat/lng
df_all.dropna(subset=['end_lat','end_lng'], inplace=True)

In [64]:
# Replace missing station names with 'Unknown'
df_all.start_station_name.fillna('Unknown', inplace=True)
df_all.end_station_name.fillna('Unknown', inplace=True)

In [65]:
# Check for missing values again
df_all.isna().sum()

rideable_type         0
started_at            0
ended_at              0
start_station_name    0
end_station_name      0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
duration              0
duration_mins         0
weekday               0
dtype: int64

In [69]:
# Check for value counts to make sure 'Unknown' makes sense
df_all.start_station_name.value_counts()

Unknown                            282014
Streeter Dr & Grand Ave             58006
Lake Shore Dr & Monroe St           43652
Theater on the Lake                 39468
Clark St & Elm St                   38521
                                    ...  
Avenue L & 114th St                     2
N Damen Ave & W Wabansia St             1
N Hampden Ct & W Diversey Ave           1
S Michigan Ave & E 118th St             1
Lyft Driver Center Private Rack         1
Name: start_station_name, Length: 713, dtype: int64

In [70]:
df_all.end_station_name.value_counts()

Unknown                            309587
Streeter Dr & Grand Ave             60558
Lake Shore Dr & Monroe St           42567
Theater on the Lake                 41246
Lake Shore Dr & North Blvd          40550
                                    ...  
Leavitt St & Division St (*)            3
hubbard_test_lws                        2
S Aberdeen St & W 106th St              2
N Hampden Ct & W Diversey Ave           1
Lyft Driver Center Private Rack         1
Name: end_station_name, Length: 714, dtype: int64

The Data is now clean! There is more that could be done in regards to the station names (as we can see above, that some instances only appear once or twice) which do appear quite peculiar. But as that is not essential to our future analysis, we will leave it be for the time being.

#### Save DataFrame

In [71]:
# Save to CSV
df_all.to_csv('data/yearly_clean.csv')

In [72]:
# Save to Pickle
df_all.to_pickle('data/yearly_clean.pkl')